In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Read the admission file
file_path = 'admissions.csv'
df = pd.read_csv(file_path)
# Group the data by suject_id and get the count of each suject_id

grouped_data = df.groupby('subject_id').size().reset_index(name='Frequency')

# Merge the grouped data back to the original DataFrame on 'subject_id'
df = pd.merge(df, grouped_data, on='subject_id', how='left')

# change the data type of admittime and dischtime from object to date and time
df['admittime'] = pd.to_datetime(df['admittime'])
df['dischtime'] = pd.to_datetime(df['dischtime'])

df.head()


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,Frequency
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,5/6/2180 19:17,5/6/2180 23:30,0,4
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,6/26/2180 15:54,6/26/2180 21:31,0,4
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,8/5/2180 20:58,8/6/2180 1:44,0,4
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,7/23/2180 5:54,7/23/2180 14:00,0,4
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,3/3/2160 21:55,3/4/2160 6:26,0,1


In [3]:
# Sort and group the admitime corresponding to subject_id
df.sort_values(by=['subject_id','admittime'], inplace=True)

visit_frequency_data = []

# Group the data by 'subject_id'
grouped_data = df.groupby('subject_id')

# Iterate over each subject_id
for subject_id, group_df in grouped_data:
    # Check if the frequency of visits for this subject_id is greater than 1
    if len(group_df) > 2:
        # Iterate over each row in the group DataFrame
        for i in range(len(group_df)-1):
            # Calculate the visit frequency
            visit_fre = group_df.admittime.iloc[i+1] - group_df.dischtime.iloc[i]
            dis_adm_typ = group_df.admission_type.iloc[i]
            readm_typ = group_df.admission_type.iloc[i+1] 
            adm_id = group_df.hadm_id.iloc[i+1]
            # Append the results to the list as a dictionary
            visit_frequency_data.append({
                'Subject_ID': subject_id,
                'Adm_id':adm_id,
                'Visit_Frequency': visit_fre,
                'Discharge_adm_type': dis_adm_typ,
                'Readm_type': readm_typ                
                
            })

# Convert the list of dictionaries to a DataFrame
visit_frequency_df = pd.DataFrame(visit_frequency_data)

# Display the first 10 rows of the DataFrame
print(visit_frequency_df.head(10))
visit_frequency_df.to_excel('Admission_type_VI.xlsx', sheet_name = 'Data')

   Subject_ID    Adm_id    Visit_Frequency Discharge_adm_type      Readm_type
0    10000032  22841357   50 days 01:12:00             URGENT        EW EMER.
1    10000032  29079034   25 days 17:46:00           EW EMER.        EW EMER.
2    10000032  25742920   11 days 05:49:00           EW EMER.        EW EMER.
3    10000826  21086876    6 days 01:09:00           EW EMER.    DIRECT EMER.
4    10000826  28289260    6 days 04:48:00       DIRECT EMER.        EW EMER.
5    10000935  24955974    2 days 16:52:00           EW EMER.  EU OBSERVATION
6    10000935  21738619 1342 days 01:47:00     EU OBSERVATION  EU OBSERVATION
7    10000935  26381316   42 days 10:07:00     EU OBSERVATION        EW EMER.
8    10000935  25849114   44 days 03:34:00           EW EMER.        EW EMER.
9    10000980  26913865  538 days 14:08:00           EW EMER.        EW EMER.


In [69]:
def visit_interval_files(input_data, i):
    """
    Filter the DataFrame to get rows where 'Visit_Frequency_days' is within the specified range,
    and save the resulting DataFrame to an Excel file.

    Parameters:
    - input_file: DataFrame, the input DataFrame containing visit frequency data
    - i: int, the lower bound of the visit frequency range
    """
    try:
        input_file = pd.read_excel(input_data)
        # Filter the DataFrame to get rows where 'Visit_Frequency_days' is within the specified range
        output_file = input_file[(input_file['Visit_Frequency'] >= i) & 
                                 (input_file['Visit_Frequency'] < i+1)]

        # Display the resulting DataFrame
        # print(output_file)
        
        # Save the filtered DataFrame to an Excel file
        output_file.to_excel(f"visit_interval_{i}_{i+1}.xlsx", sheet_name='Data', index=False)
        
        print(f"Filtered data saved successfully to visit_interval_{i}_{i+1}.xlsx")
    except Exception as e:
        print("Error occurred:", str(e))


In [70]:
def same_admission_type(input_data, j):
    """
    Identify cases where the discharge admission type matches the readmission type,
    and save the counts of each admission type to an Excel file.

    Parameters:
    - input_file: DataFrame, the input DataFrame containing discharge and readmission types
    - j: int, the lower bound of the range used for naming the output Excel file
    """
    try:
        input_file = pd.read_excel(input_data)
        # Create an empty list to store the admission types
        same_adm_type = []

        # Iterate through each row of the DataFrame
        for i in range(len(input_file)):
            # Get the discharge admission type and readmission type for the current row
            discharge_adm_type = input_file['Discharge_adm_type'].iloc[i]
            readm_type = input_file['Readm_type'].iloc[i]
            Subject_ID = input_file['Subject_ID'].iloc[i]
            Adm_id = input_file['Adm_id'].iloc[i]
            # Check if the discharge admission type and readmission type are equal
            if discharge_adm_type == readm_type:
                # If they are equal, append the discharge admission type to the list
                same_adm_type.append({
                         'Adm_type':discharge_adm_type,
                         'subject_id':Subject_ID,
                         'hadm_id': Adm_id,
                         })
        
        # Convert list to DataFrame
        output_file = pd.DataFrame(same_adm_type)  
        # print(output_file)

        # Count occurrences of each admission type
        # Ad_VI = output_file['Admission_Type'].value_counts()
        # print(Ad_VI)

        # Save the counts to an Excel file
        output_file.to_excel(f"Adm_type_VI_{j}_{j+1}.xlsx")
        
        print(f"Filtered data saved successfully to Adm_type_VI_{j}_{j+1}.xlsx")
    except Exception as e:
        print("Error occurred:", str(e))


In [185]:
def discharge_notes (input_file, adm_type, j):
    try:
        # Assuming 'dis_data' is available in the global scope
        discharge_data = dis_data[['subject_id', 'hadm_id', 'text']]
        
        # Read admission data from the specified Excel file
        adm_data = pd.read_excel(input_file)
        
        # Merge discharge data with admission data
        merged_df = pd.merge(discharge_data, adm_data, on='hadm_id', how='right')
        
        # Select rows with a specific admission type
        selected_rows = merged_df[merged_df['Adm_type'] == adm_type]
        selected_columns = selected_rows[['subject_id_y','hadm_id','Adm_type','text']]
        # Drop rows with null values
        output_file = selected_columns.dropna()
        
        # Save the filtered data to an Excel file
        output_file.to_excel(f"discharge_notes_{j}_{j+1}.xlsx", index=False)
        print(f"Filtered data saved successfully to discharge_notes_{j}_{j+1}.xlsx")
    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found.")
    except Exception as e:
        print("Error occurred:", str(e))


In [74]:
dis_data = pd.read_csv('discharge.csv')
discharge_data = dis_data[['subject_id', 'hadm_id', 'text']]

In [107]:
visit_interval_files('Admission_type_VI.xlsx',0)

Filtered data saved successfully to visit_interval_0_1.xlsx


In [108]:
same_admission_type('visit_interval_0_1.xlsx', 0)

Filtered data saved successfully to Adm_type_VI_0_1.xlsx


In [186]:
discharge_notes ('Adm_type_VI_0_1.xlsx','EU OBSERVATION',0)

Filtered data saved successfully to discharge_notes_0_1.xlsx


In [110]:
visit_interval_files('Admission_type_VI.xlsx',1)

Filtered data saved successfully to visit_interval_1_2.xlsx


In [111]:
same_admission_type('visit_interval_1_2.xlsx', 1)

Filtered data saved successfully to Adm_type_VI_1_2.xlsx


In [187]:
discharge_notes ('Adm_type_VI_1_2.xlsx','EW EMER.',1)

Filtered data saved successfully to discharge_notes_1_2.xlsx


In [113]:
visit_interval_files('Admission_type_VI.xlsx',2)

Filtered data saved successfully to visit_interval_2_3.xlsx


In [114]:
same_admission_type('visit_interval_2_3.xlsx', 2)

Filtered data saved successfully to Adm_type_VI_2_3.xlsx


In [188]:
discharge_notes ('Adm_type_VI_2_3.xlsx','EW EMER.',2)

Filtered data saved successfully to discharge_notes_2_3.xlsx


In [122]:
visit_interval_files('Admission_type_VI.xlsx',3)

Filtered data saved successfully to visit_interval_3_4.xlsx


In [123]:
same_admission_type('visit_interval_3_4.xlsx', 3)

Filtered data saved successfully to Adm_type_VI_3_4.xlsx


In [189]:
discharge_notes ('Adm_type_VI_3_4.xlsx','EW EMER.',3)

Filtered data saved successfully to discharge_notes_3_4.xlsx


In [125]:
visit_interval_files('Admission_type_VI.xlsx',4)

Filtered data saved successfully to visit_interval_4_5.xlsx


In [126]:
same_admission_type('visit_interval_4_5.xlsx', 4)

Filtered data saved successfully to Adm_type_VI_4_5.xlsx


In [190]:
discharge_notes ('Adm_type_VI_4_5.xlsx','EW EMER.',4)

Filtered data saved successfully to discharge_notes_4_5.xlsx


In [128]:
visit_interval_files('Admission_type_VI.xlsx',5)

Filtered data saved successfully to visit_interval_5_6.xlsx


In [130]:
same_admission_type('visit_interval_5_6.xlsx', 5)

Filtered data saved successfully to Adm_type_VI_5_6.xlsx


In [191]:
discharge_notes ('Adm_type_VI_5_6.xlsx','EW EMER.',5)

Filtered data saved successfully to discharge_notes_5_6.xlsx


In [140]:
visit_interval_files('Admission_type_VI.xlsx',6)

Filtered data saved successfully to visit_interval_6_7.xlsx


In [141]:
same_admission_type('visit_interval_6_7.xlsx', 6)

Filtered data saved successfully to Adm_type_VI_6_7.xlsx


In [192]:
discharge_notes ('Adm_type_VI_6_7.xlsx','EW EMER.',6)

Filtered data saved successfully to discharge_notes_6_7.xlsx


In [143]:
visit_interval_files('Admission_type_VI.xlsx',7)

Filtered data saved successfully to visit_interval_7_8.xlsx


In [144]:
same_admission_type('visit_interval_7_8.xlsx', 7)

Filtered data saved successfully to Adm_type_VI_7_8.xlsx


In [193]:
discharge_notes ('Adm_type_VI_7_8.xlsx','EW EMER.',7)

Filtered data saved successfully to discharge_notes_7_8.xlsx


In [146]:
visit_interval_files('Admission_type_VI.xlsx',8)

Filtered data saved successfully to visit_interval_8_9.xlsx


In [147]:
same_admission_type('visit_interval_8_9.xlsx', 8)

Filtered data saved successfully to Adm_type_VI_8_9.xlsx


In [194]:
discharge_notes ('Adm_type_VI_8_9.xlsx','EW EMER.',8)

Filtered data saved successfully to discharge_notes_8_9.xlsx


In [149]:
visit_interval_files('Admission_type_VI.xlsx',9)

Filtered data saved successfully to visit_interval_9_10.xlsx


In [150]:
same_admission_type('visit_interval_9_10.xlsx', 9)

Filtered data saved successfully to Adm_type_VI_9_10.xlsx


In [195]:
discharge_notes ('Adm_type_VI_9_10.xlsx','EW EMER.',9)

Filtered data saved successfully to discharge_notes_9_10.xlsx


In [152]:
visit_interval_files('Admission_type_VI.xlsx',10)

Filtered data saved successfully to visit_interval_10_11.xlsx


In [153]:
same_admission_type('visit_interval_10_11.xlsx', 10)

Filtered data saved successfully to Adm_type_VI_10_11.xlsx


In [196]:
discharge_notes ('Adm_type_VI_10_11.xlsx','EW EMER.',10)

Filtered data saved successfully to discharge_notes_10_11.xlsx


In [155]:
visit_interval_files('Admission_type_VI.xlsx',11)

Filtered data saved successfully to visit_interval_11_12.xlsx


In [156]:
same_admission_type('visit_interval_11_12.xlsx', 11)

Filtered data saved successfully to Adm_type_VI_11_12.xlsx


In [197]:
discharge_notes ('Adm_type_VI_11_12.xlsx','EW EMER.',11)

Filtered data saved successfully to discharge_notes_11_12.xlsx


In [162]:
visit_interval_files('Admission_type_VI.xlsx',12)

Filtered data saved successfully to visit_interval_12_13.xlsx


In [163]:
same_admission_type('visit_interval_12_13.xlsx', 12)

Filtered data saved successfully to Adm_type_VI_12_13.xlsx


In [198]:
discharge_notes ('Adm_type_VI_12_13.xlsx','EW EMER.',12)

Filtered data saved successfully to discharge_notes_12_13.xlsx


In [165]:
visit_interval_files('Admission_type_VI.xlsx',13)

Filtered data saved successfully to visit_interval_13_14.xlsx


In [166]:
same_admission_type('visit_interval_13_14.xlsx', 13)

Filtered data saved successfully to Adm_type_VI_13_14.xlsx


In [199]:
discharge_notes ('Adm_type_VI_13_14.xlsx','EW EMER.',13)

Filtered data saved successfully to discharge_notes_13_14.xlsx


In [168]:
visit_interval_files('Admission_type_VI.xlsx',14)

Filtered data saved successfully to visit_interval_14_15.xlsx


In [169]:
same_admission_type('visit_interval_14_15.xlsx', 14)

Filtered data saved successfully to Adm_type_VI_14_15.xlsx


In [200]:
discharge_notes ('Adm_type_VI_14_15.xlsx','EW EMER.',14)

Filtered data saved successfully to discharge_notes_14_15.xlsx
